In [1]:
import pandas as pd
import numpy as np
import cassiopeia as cas
import seaborn as sns
from os.path import join, exists
from os import makedirs, getcwd
from pandarallel import pandarallel
import networkx as nx
import matplotlib.pyplot as plt
from networkx.drawing.nx_agraph import write_dot, graphviz_layout

pandarallel.initialize(nb_workers=32)

import pickle

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
af_f = "/data2/mito_lineage/data/processed/mttrace/TcellDupi_may17_2021/MTblacklist/post/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/af_by_cell.tsv"
dp_f = "/data2/mito_lineage/data/processed/mttrace/TcellDupi_may17_2021/MTblacklist/post/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/af_by_cell.DP.tsv"
ad_f = "/data2/mito_lineage/data/processed/mttrace/TcellDupi_may17_2021/MTblacklist/post/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/af_by_cell.AD.tsv"
donors = "/data2/mito_lineage/data/processed/mttrace/TcellDupi_may17_2021/MTblacklist/merged/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/filter_mgatk/vireoIn/multiplex/cells_meta.tsv"
prefix= "TcellDupi_may17_2021/MTblacklist"
name = "post_cass_test_missingV"
outdir = "./output/data/"
af_thresh = 0.01
dp_thresh = 2

In [3]:
outdir = join(outdir, prefix)
if not exists(outdir):
    print(f"Making outdir {outdir} in folder {getcwd()}")
    makedirs(outdir)

In [4]:
af = pd.read_csv(af_f, index_col=0, sep='\t')
dp = pd.read_csv(dp_f, index_col=0, sep='\t').astype(int)
ad = pd.read_csv(ad_f, index_col=0, sep='\t').astype(int)


## Create character matrix - binarize af by af_thresh, and set any element under dp_thresh to -1

In [5]:
#Binarize
af = af.applymap(lambda x: 0 if x<af_thresh else 1)
af

,1040C,10634C,10646T,1082G,10912A,1095C,11123A,11137C,11175C,11204A,...,9531G,954G,9590T,9640T,9658G,971C,9750A,9823A,983T,9861G
Cell,,,,,,,,,,,,,,,,,,,,,
AAACGAAAGAACAGGA-1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
AAACGAAAGATTGACA-1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
AAACGAAAGCGTTGCC-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACGAAAGGTACTCT-1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
AAACGAAAGGTTAACA-1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTGTTCCGGAAAG-1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
TTTGTGTTCCTGACCT-1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
TTTGTGTTCGGATGTT-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
def dp_where(x, dp, dp_thresh):
#    print('name', x.name)
    curr = dp.loc[x.name]<dp_thresh
    x.loc[curr] = -1 #"-"
    return x
af = af.parallel_apply(dp_where, axis=1, args=(dp, dp_thresh))

In [7]:
character_matrix = af.copy()
#character_matrix = af.parallel_apply(dp_where, axis=1, args=(dp,dp_thresh))
character_matrix

,1040C,10634C,10646T,1082G,10912A,1095C,11123A,11137C,11175C,11204A,...,9531G,954G,9590T,9640T,9658G,971C,9750A,9823A,983T,9861G
Cell,,,,,,,,,,,,,,,,,,,,,
AAACGAAAGAACAGGA-1,-1,0,-1,-1,-1,-1,-1,-1,-1,-1,...,1,-1,-1,-1,-1,-1,-1,-1,-1,-1
AAACGAAAGATTGACA-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,-1,-1,-1,-1,-1,-1,-1,-1,-1
AAACGAAAGCGTTGCC-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
AAACGAAAGGTACTCT-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,-1,-1,-1,-1,-1,-1,-1,1,-1
AAACGAAAGGTTAACA-1,-1,-1,-1,-1,-1,-1,1,0,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTGTTCCGGAAAG-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,-1,-1,-1,-1,-1,-1,-1,-1,-1
TTTGTGTTCCTGACCT-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,-1,-1,-1,-1,-1,-1,-1,-1,-1
TTTGTGTTCGGATGTT-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


## Each cell has at least one variant, and each variant seen in each cell (following 2 lines)

In [8]:
((character_matrix==0) | (character_matrix==-1)).all(axis=0).any()

False

In [9]:
((character_matrix==0) | (character_matrix==-1)).all(axis=1).any()

False

## Rename variants

In [10]:
var_map = {val:f"r{i}" for i, val in enumerate(character_matrix.columns) }
character_matrix = character_matrix.rename(var_map, axis=1)

In [11]:
character_matrix.to_csv(join(outdir, "missing.csv"))

## Reconstructing Trees

In [12]:
priors=None
if priors is not None:
    cas_tree = cas.data.CassiopeiaTree(character_matrix=character_matrix, priors=priors)
else:
    cas_tree = cas.data.CassiopeiaTree(character_matrix=character_matrix)
cas_tree.character_matrix.head(5)


,r0,r1,r2,r3,r4,r5,r6,r7,r8,r9,...,r258,r259,r260,r261,r262,r263,r264,r265,r266,r267
Cell,,,,,,,,,,,,,,,,,,,,,
AAACGAAAGAACAGGA-1,-1,0,-1,-1,-1,-1,-1,-1,-1,-1,...,1,-1,-1,-1,-1,-1,-1,-1,-1,-1
AAACGAAAGATTGACA-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,-1,-1,-1,-1,-1,-1,-1,-1,-1
AAACGAAAGCGTTGCC-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
AAACGAAAGGTACTCT-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,-1,-1,-1,-1,-1,-1,-1,1,-1
AAACGAAAGGTTAACA-1,-1,-1,-1,-1,-1,-1,1,0,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [13]:
cas_tree.n_cell, cas_tree.n_character,


(10051, 268)

In [14]:
# cell_meta = clone_allele_table.groupby('cellBC').agg({"intBC": 'nunique', 'UMI': 'sum', 'sampleID': 'unique'})
# cell_meta['sampleID'] = [x[0] for x in cell_meta['sampleID']]

# missing_proportion = (character_matrix == -1).sum(axis=0) / character_matrix.shape[0]
# uncut_proportion = (character_matrix == 0).sum(axis=0) / character_matrix.shape[0]
# n_unique_states = character_matrix.apply(lambda x: len(np.unique(x[(x != 0) & (x != -1)])), axis=0)

# character_meta = pd.DataFrame([missing_proportion, uncut_proportion, n_unique_states], index = ['missing_prop', 'uncut_prop', 'n_unique_states']).T

# cas_tree.cell_meta = cell_meta
# cas_tree.character_meta = character_meta

## HybridSolver 
1. Top-down greedy solver followed by
2. Bottom-solve of any tipe. 

In [15]:
# REINSTANTIATE the bottom and top solvers
vanilla_greedy = cas.solver.VanillaGreedySolver()
if priors is not None:
    ilp_solver = cas.solver.ILPSolver(convergence_time_limit=12600, maximum_potential_graph_layer_size=10000, weighted=True, seed=1234)
else:
    ilp_solver = cas.solver.ILPSolver(convergence_time_limit=12600, maximum_potential_graph_layer_size=10000, weighted=False, seed=1234)


In [ ]:
hybrid_solver = cas.solver.HybridSolver(top_solver=vanilla_greedy, bottom_solver=ilp_solver, cell_cutoff=40, threads=24)
hybrid_solver.solve(cas_tree, logfile='example_hybrid.log')


example_hybrid.log
None
subtree 0


[2021-08-12 23:18:44,210]    INFO [ILPSolver] Solving tree with the following parameters.
[2021-08-12 23:18:44,214]    INFO [ILPSolver] Convergence time limit: 12600
[2021-08-12 23:18:44,217]    INFO [ILPSolver] Convergence iteration limit: 0
[2021-08-12 23:18:44,220]    INFO [ILPSolver] Max potential graph layer size: 10000
[2021-08-12 23:18:44,223]    INFO [ILPSolver] Max potential graph lca distance: None
[2021-08-12 23:18:44,227]    INFO [ILPSolver] MIP gap: 0.01


subtree 0


[2021-08-12 23:18:44,284]    INFO [ILPSolver] Solving tree with the following parameters.
[2021-08-12 23:18:44,288]    INFO [ILPSolver] Convergence time limit: 12600
[2021-08-12 23:18:44,290]    INFO [ILPSolver] Convergence iteration limit: 0
[2021-08-12 23:18:44,293]    INFO [ILPSolver] Max potential graph layer size: 10000
[2021-08-12 23:18:44,296]    INFO [ILPSolver] Max potential graph lca distance: None
[2021-08-12 23:18:44,298]    INFO [ILPSolver] MIP gap: 0.01


subtree 0


...

  File "/data/isshamie/software/anaconda3/envs/mito_trace/lib/python3.6/site-packages/ngs_tools/logging.py", line 62, in inner
    return func(*args, **kwargs)
  File "/data/isshamie/software/anaconda3/envs/mito_trace/lib/python3.6/site-packages/cassiopeia/solver/ILPSolver.py", line 212, in solve
    cassiopeia_tree.missing_state_indicator,
  File "/data/isshamie/software/anaconda3/envs/mito_trace/lib/python3.6/site-packages/cassiopeia/solver/ILPSolver.py", line 304, in infer_potential_graph
    missing_state_indicator,
  File "/data/isshamie/software/anaconda3/envs/mito_trace/lib/python3.6/site-packages/ngs_tools/logging.py", line 111, in info
    self.namespace_message(message), *args, **kwargs
Message: '[ILPSolver] (Process: fcce2bb71fbd410caeda23bd403f9f7a) LCA distance 6 completed with a neighborhood size of 1749.'
Arguments: ()
--- Logging error ---
Traceback (most recent call last):
  File "/data/isshamie/software/anaconda3/envs/mito_trace/lib/python3.6/logging/__init__.py", lin

  File "/data/isshamie/software/anaconda3/envs/mito_trace/lib/python3.6/multiprocessing/context.py", line 277, in _Popen
    return Popen(process_obj)
  File "/data/isshamie/software/anaconda3/envs/mito_trace/lib/python3.6/multiprocessing/popen_fork.py", line 19, in __init__
    self._launch(process_obj)
  File "/data/isshamie/software/anaconda3/envs/mito_trace/lib/python3.6/multiprocessing/popen_fork.py", line 73, in _launch
    code = process_obj._bootstrap()
  File "/data/isshamie/software/anaconda3/envs/mito_trace/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/data/isshamie/software/anaconda3/envs/mito_trace/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/data/isshamie/software/anaconda3/envs/mito_trace/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/data/isshamie/software/anaconda3/envs/mito_trace/lib/python3.6/

  File "/data/isshamie/software/anaconda3/envs/mito_trace/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3263, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):
  File "/data/isshamie/software/anaconda3/envs/mito_trace/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-16-82d74439f152>", line 2, in <module>
    hybrid_solver.solve(cas_tree, logfile='example_hybrid.log')
  File "/data/isshamie/software/anaconda3/envs/mito_trace/lib/python3.6/site-packages/cassiopeia/solver/HybridSolver.py", line 147, in solve
    with multiprocessing.Pool(processes=self.threads) as pool:
  File "/data/isshamie/software/anaconda3/envs/mito_trace/lib/python3.6/multiprocessing/context.py", line 119, in Pool
    context=self.get_context())
  File "/data/isshamie/software/anaconda3/envs/mito_trace/lib/python3.6/multiprocessing/pool.py", line 174, in __in

  File "/data/isshamie/software/anaconda3/envs/mito_trace/lib/python3.6/site-packages/contextvars/__init__.py", line 38, in run
    return callable(*args, **kwargs)
  File "/data/isshamie/software/anaconda3/envs/mito_trace/lib/python3.6/site-packages/ipykernel/ipkernel.py", line 302, in do_execute
    res = shell.run_cell(code, store_history=store_history, silent=silent)
  File "/data/isshamie/software/anaconda3/envs/mito_trace/lib/python3.6/site-packages/ipykernel/zmqshell.py", line 539, in run_cell
    return super(ZMQInteractiveShell, self).run_cell(*args, **kwargs)
  File "/data/isshamie/software/anaconda3/envs/mito_trace/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2867, in run_cell
    raw_cell, store_history, silent, shell_futures)
  File "/data/isshamie/software/anaconda3/envs/mito_trace/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2895, in _run_cell
    return runner(coro)
  File "/data/isshamie/software/anaconda3/envs/mito_trace/l

  File "/data/isshamie/software/anaconda3/envs/mito_trace/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 358, in process_one
    yield gen.maybe_future(dispatch(*args))
  File "/data/isshamie/software/anaconda3/envs/mito_trace/lib/python3.6/site-packages/tornado/gen.py", line 234, in wrapper
    yielded = ctx_run(next, result)
  File "/data/isshamie/software/anaconda3/envs/mito_trace/lib/python3.6/site-packages/contextvars/__init__.py", line 38, in run
    return callable(*args, **kwargs)
  File "/data/isshamie/software/anaconda3/envs/mito_trace/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 261, in dispatch_shell
    yield gen.maybe_future(handler(stream, idents, msg))
  File "/data/isshamie/software/anaconda3/envs/mito_trace/lib/python3.6/site-packages/tornado/gen.py", line 234, in wrapper
    yielded = ctx_run(next, result)
  File "/data/isshamie/software/anaconda3/envs/mito_trace/lib/python3.6/site-packages/contextvars/__init__.py", line 38, in run
    retur

In [ ]:
character_matrix.duplicated().any()

In [ ]:
character_matrix.duplicated()

In [ ]:
!ls | grep 'example_hybrid'


In [ ]:
rndict = {}
_iter = 0
for n in cas_tree.nodes:
    if ',' in n:
        rndict[n] = f'node{_iter}'
        _iter += 1

cas_tree.relabel_nodes(rndict)

In [ ]:
pickle.dump(cas_tree, open(join(outdir, name+".full.missingV.castree.p"), 'wb'))

# Plot tree 

## Plot with networkx


In [ ]:
G = nx.DiGraph()
G.add_nodes_from(cas_tree.nodes)
G.add_edges_from(cas_tree.edges)
G

# write dot file to use with graphviz
# run "dot -Tpng test.dot >test.png"

f = plt.figure(figsize=(10,10), dpi=300)
write_dot(G,join(outdir, f'{name}_hybrid.dot'))


# same layout using matplotlib with no labels
plt.title('draw_networkx')
pos = graphviz_layout(G, prog='dot', root=cas_tree.root)
nx.draw(G, pos, with_labels=False, arrows=True)

# color_map = []
# for node in G:
#     if node < 10:
#         color_map.append('blue')
#     else: 
#         color_map.append('green')
# nx.draw(G, pos, node_color=color_map, with_labels=False)


plt.savefig(join(outdir, f'{name}_hybrid.png'))


## Plot with itol (only with valid itol account)

In [ ]:
# plot tree
cas.pl.upload_and_export_itol(cas_tree, itol_config="~/.itolconfig_hidden",
                              tree_name = "example_hybrid_tree", export_filepath="example_hybrid_tree.png")


## Test a sample of the data

In [ ]:
priors=None
if priors is not None:
    cas_tree = cas.data.CassiopeiaTree(character_matrix=character_matrix.iloc[:,:50], priors=priors)
else:
    cas_tree = cas.data.CassiopeiaTree(character_matrix=character_matrix.iloc[:,:50])
# REINSTANTIATE the bottom and top solvers
vanilla_greedy = cas.solver.VanillaGreedySolver()
if priors is not None:
    ilp_solver = cas.solver.ILPSolver(convergence_time_limit=500, maximum_potential_graph_layer_size=500, weighted=True, seed=1234)
else:
    ilp_solver = cas.solver.ILPSolver(convergence_time_limit=500, maximum_potential_graph_layer_size=500, weighted=False, seed=1234)

hybrid_solver = cas.solver.HybridSolver(top_solver=vanilla_greedy, bottom_solver=ilp_solver, cell_cutoff=40, threads=10)
hybrid_solver.solve(cas_tree, logfile='example_hybrid.log')


In [ ]:
G = nx.DiGraph()
G.add_nodes_from(cas_tree.nodes)
G.add_edges_from(cas_tree.edges)
G

# write dot file to use with graphviz
# run "dot -Tpng test.dot >test.png"

f = plt.figure(figsize=(10,10), dpi=300)
write_dot(G,join(outdir, f'{name}_hybrid.dot'))

# same layout using matplotlib with no labels
plt.title('draw_networkx')
pos = graphviz_layout(G, prog='dot', root=cas_tree.root)
nx.draw(G, pos, with_labels=False, arrows=True)

plt.savefig(join(outdir, f'{name}_hybrid.png'))
